In [ ]:
%matplotlib inline

import csv
import xlrd
import pandas as pd
import seaborn as sns
from requests import get, session
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

In [ ]:
data = []
parties = {
    'CLP': 'LNP',
    'LP': 'LNP',
    'NP': 'LNP',
    'ALP': 'ALP',
    'Australian Labor Party': 'ALP',
    'Liberal Party': 'LNP',
    'National Party': 'LNP',
    'Country Liberal Party (NT)': 'LNP'
}
tpp = {
    'LNP': 0,
    'ALP': 0
}

def tpp_margin(seats_won, seats_total):
    return round((seats_won / seats_total) * 100, 2)


def tpp_from_csv(csv_data: str, coalition: list, alp: str):
    lnp = 0
    lab = 0
    lines = csv_data.splitlines()
    reader = csv.reader(lines, delimiter=',')
    for row in reader:
        if row[0] in alp:
            lab = int(row[-2])
        elif row[0] in coalition:
            lnp += int(row[-2])
    return lnp, lab


def loop_excel_column(excel_file):
    _tpp = dict(**tpp)
    book = xlrd.open_workbook(excel_file)
    sheet = book.sheet_by_index(1)
    for _ in range(5, 153):
        x = sheet.cell(_, 3).value
        try:
            _tpp[parties[x]] += 1
        except KeyError:
            pass
    return _tpp


def create_row(tpp_data: dict[str, int], seats: int, year: int, election: int):
    return [
        year,
        tpp_data['LNP'],
        tpp_data['ALP'],
        seats,
        tpp_margin(tpp_data['LNP'], seats),
        tpp_margin(tpp_data['ALP'], seats),
        election
    ]


def read_aec_csv(csv_url):
    tpp_data = dict(**tpp)
    tpp_data['LNP'], tpp_data['ALP'] = tpp_from_csv(get(csv_url).text, coalition=['LP', 'NP', 'CLP', 'LNQ', 'Liberal', 'Liberal National Party', 'The Nationals', 'Country Liberals (NT)'], alp=['ALP', 'Australian Labor Party'])
    return tpp_data


def read_table_from_parl_html(year: int, markup: BeautifulSoup):
    _tpp = dict(**tpp)
    for title in markup.find_all('h3'):
        if str(year) in title.text:
            table = title.findNext('table')
    ended = False
    for row in table.find_all('tr'):
        if 'Seats Won' in row.text:
            while not ended:
                row = row.findNext('tr')
                if 'Total' in row.text:
                    seats = int(row.find_all('td')[-1].text.strip())
                    ended = True
                else:
                    try:
                        _tpp[parties[row.find('td').text.strip().replace('(a)', '')]] += int(row.find_all('td')[-1].text.strip())
                    except KeyError:
                        pass
    return _tpp, seats


def find_election_table(year, wa_page):
    for election_year in wa_page.find_all('div', {'class': 'col-xs-3 elborders'}):
        if str(year) in election_year.text:
            return 'https://elections.uwa.edu.au/' + election_year.find('a')['href']


def scrape_table(url, session):
    _tpp = dict(**tpp)
    table = BeautifulSoup(session.get(url).text, 'lxml').find('div', {'id': 'votes'}).find('tbody')
    for row in table.find_all('tr'):
        try:
            _tpp[parties[row.find('td').text.strip()]] += int(row.find_all('td')[4].text.strip())
        except KeyError:
            pass
    return _tpp, int(table.find_all('tr')[-1].find_all('td')[4].text.strip())

In [ ]:
# Elections prior to 1993
data_report = BeautifulSoup(get('https://www.aph.gov.au/About_Parliament/Parliamentary_Departments/Parliamentary_Library/pubs/rp/rp9899/99RP08').text, 'lxml')


rs = session()
rs.get('https://elections.uwa.edu.au/index.lasso')
wa_uni_data = BeautifulSoup(rs.get('https://elections.uwa.edu.au/listelections.lasso?ElectionType=2').text, 'lxml').find_all('div', {'class': 'row elrow elbordersides'})[1]

In [ ]:
# 1972
_tpp, seats = scrape_table(find_election_table(1972, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1972,
    1
))

In [ ]:
# 1974
_tpp, seats = scrape_table(find_election_table(1974, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1974,
    2
))

In [ ]:
# 1975
_tpp, seats = scrape_table(find_election_table(1975, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1975,
    1
))

In [ ]:
# 1977
_tpp, seats = scrape_table(find_election_table(1977, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1977,
    2
))

In [ ]:
# 1980
_tpp, seats = scrape_table(find_election_table(1980, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1980,
    3
))

In [ ]:
#1983
_tpp, seats = read_table_from_parl_html(1983, data_report)
data.append(create_row(
    _tpp,
    seats,
    1983,
    1
))

In [ ]:
#1984
_tpp, seats = read_table_from_parl_html(1984, data_report)
data.append(create_row(
    _tpp,
    seats,
    1984,
    2
))

In [ ]:
# 1987
_tpp, seats = scrape_table(find_election_table(1987, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1987,
    3
))

In [ ]:
# 1990
_tpp, seats = scrape_table(find_election_table(1990, wa_uni_data), rs)
data.append(create_row(
    _tpp,
    seats,
    1990,
    4
))

In [ ]:
# 1993
data.append(create_row(
    loop_excel_column('data/1993/MEMB93.XLS'),
    148,
    1993,
    5
))

In [ ]:
# 1996
data.append(create_row(
    loop_excel_column('data/1996/MEMB96.XLS'),
    148,
    1996,
    1
))

In [ ]:
# 1998
data.append(create_row(
    loop_excel_column('data/1998/MEMB98.XLS'),
    148,
    1998,
    2
))

In [ ]:
# 2001

tpp_2001 = dict(**tpp)

workbook = xlrd.open_workbook('data/2001/Partyrep.xls')
sheet = workbook.sheet_by_index(1)

tpp_2001['LNP'] = int(sheet.cell(14,1).value + sheet.cell(14,2).value + sheet.cell(14,3).value)
tpp_2001['ALP'] = int(sheet.cell(14,4).value)
data.append(create_row(
    tpp_2001,
    150,
    2001,
    3
))

In [ ]:
# 2004
data.append(create_row(
    read_aec_csv('https://results.aec.gov.au/12246/results/Downloads/HousePartyRepresentationDownload-12246.csv'),
    150,
    2004,
    4
))

In [ ]:
# 2007
data.append(create_row(
    read_aec_csv('https://results.aec.gov.au/13745/Website/Downloads/HousePartyRepresentationDownload-13745.csv'),
    150,
    2007,
    1
))

In [ ]:
# 2010
data.append(create_row(
    read_aec_csv('https://results.aec.gov.au/15508/Website/Downloads/HousePartyRepresentationDownload-15508.csv'),
    150,
    2010,
    2
))

In [ ]:
# 2013
data.append(create_row(
    read_aec_csv('https://results.aec.gov.au/17496/Website/Downloads/HousePartyRepresentationLeadingDownload-17496.csv'),
    150,
    2013,
    1
))

In [ ]:
# 2016
data.append(create_row(
    read_aec_csv('https://results.aec.gov.au/20499/Website/Downloads/HousePartyRepresentationLeadingDownload-20499.csv'),
    150,
    2016,
    2
))

In [ ]:
# 2016
data.append(create_row(
    read_aec_csv('https://results.aec.gov.au/24310/Website/Downloads/HousePartyRepresentationLeadingDownload-24310.csv'),
    151,
    2019,
    3
))

In [ ]:
df = pd.DataFrame(data=data, columns=['year', 'lnp', 'alp', 'seats', 'tpp_lnp', 'tpp_alp', 'election_index'])
df.to_csv('export.csv', index=False)
df['year'] = pd.to_datetime(df['year'], format='%Y')
df['election_index'] = df['election_index'].astype(str)

In [ ]:
fig = plt.figure()

sns.lineplot(x=df['year'], y=df['tpp_lnp'], color='blue')
sns.lineplot(x=df['year'], y=df['tpp_alp'], color='red')

plt.xlabel("Election")
plt.ylabel("LNP v ALP seat ratio")

fig.show()